In [8]:
## pre procesamiento de datos 
#leer archivos excel 


import pandas as pd

# Ruta al archivo Excel
compras_medfp = 'Source\compras_medfp.xlsx'
items_medicamentos = 'Source\items_medicamentos.xlsx'
compras_his = 'Source\ordenes_compras.xlsx'
precio_venta='Source/PrecioVenta.xlsx'


# Leer el archivo Excel
compras_medfp_df = pd.read_excel(compras_medfp, engine='openpyxl')  # Asegúrate de usar el motor 'openpyxl' para archivos .xlsx
items_medicamentos_df = pd.read_excel(items_medicamentos, engine='openpyxl') 
compras_hist_df = pd.read_excel(compras_his, engine='openpyxl') 
precio_venta_df= pd.read_excel(precio_venta,engine='openpyxl')

In [9]:
import pandas as pd
## guardar dataframes en formato parquet en carpeta raw_cl



# Save the dataframes in Parquet format
compras_medfp_df.to_parquet('raw_cl/compras_medfp.parquet')
compras_hist_df.to_parquet('raw_cl/compras_hist.parquet')
items_medicamentos_df.to_parquet('raw_cl/items_medicamentos.parquet')
precio_venta_df.to_parquet('raw_cl/precio_venta.parquet')


In [10]:
import pandas as pd

# Read Parquet files from the "raw_cl" folder
df_compras_medfp = pd.read_parquet('raw_cl/compras_medfp.parquet')
df_compras_hist = pd.read_parquet('raw_cl/compras_hist.parquet')
df_items_medicamento= pd.read_parquet('raw_cl/items_medicamentos.parquet')
df_precio_venta=pd.read_parquet('raw_cl/precio_venta.parquet')


In [11]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
from unidecode import unidecode


def quitar_tildes(valor):
    # Comprobar si el valor es una instancia de str (cadena de texto)
    if isinstance(valor, str):
        return unidecode(valor)
    # Si no es una cadena, simplemente devuelve el valor sin cambios
    return valor

# Detalles de la conexión
server = 'medpredictdb.cn2u06yialkh.us-east-2.rds.amazonaws.com'
database = 'medpredictdb'
username = 'useradm'
password = 'medpredict12345'
driver = 'ODBC Driver 18 for SQL Server'

# Codificar el nombre del controlador para su uso en URL
quoted_driver = urllib.parse.quote(driver)
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={quoted_driver}'

# Crear motor de conexión
engine = create_engine(connection_string)



In [12]:
df_compras_medfp.to_sql('ordenes_compras_med_form_pag', con=engine, schema='dbo', if_exists='replace', index=False)
df_items_medicamento.to_sql('items_medicamentos', con=engine, schema='dbo', if_exists='replace', index=False)
df_compras_hist.to_sql('compras_hitoricas', con=engine, schema='dbo', if_exists='replace', index=False)
df_precio_venta.to_sql('precio_venta', con=engine, schema='dbo', if_exists='replace', index=False)

1

In [14]:
df_items_medicamento = df_items_medicamento.rename(columns={'Descripcion': 'Descripcion_items'})

In [15]:
df_compras_medfp = df_compras_medfp.rename(columns={'Clasificacion': 'Clasificacion_fp'})
df_compras_medfp = df_compras_medfp.rename(columns={'Proveedor': 'Proveedor_fp'})

In [16]:
## prepare data in cost
compras_med = pd.merge(df_compras_hist, 
                    df_items_medicamento, 
                    left_on='Item_Commodity', 
                    right_on='Item', 
                    how='inner')

# Mostrar las primeras filas del DataFrame resultante para verificar
compras_med = compras_med[df_compras_hist.columns.to_list() +['Descripcion_items']]

In [17]:

result_df = pd.merge(compras_med, 
                     df_compras_medfp, 
                     left_on='OrdenCompra', 
                     right_on='Numero', 
                     how='left')

# Seleccionar las columnas de compras_med y añadir FormadePago y Numero de df_compras_medfp
df_ordenes_compras = result_df[compras_med.columns.tolist() + ['FormadePago', 'Numero']]


In [18]:
df_ordenes_compras.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['Companía', 'OrdenCompra', 'Item_Commodity', 'Descripcion',
       'CantidadPedida', 'CantidadRecibida', 'PrecioUnitario', 'Total',
       'Comentario', 'NEntregas', 'EstadoLínea', 'CentroCosto',
       'EstadoOrdenCompra', 'Clasificacion', 'Proveedor', 'FechaPreparación',
       'FechaEntrega', 'AlmacenIngreso', 'Ncontrato', 'UNegocio',
       'NroRequerimiento', 'CentroCosto.1', 'CentroCostoNombre',
       'RQAprobadoPor', 'Descripcion_items', 'FormadePago', 'Numero'],
      dtype='object')>

In [19]:
df_ordenes_compras=df_ordenes_compras[['Companía', 'OrdenCompra', 'Item_Commodity', 'Descripcion','Descripcion_items',
       'CantidadPedida', 'CantidadRecibida', 'PrecioUnitario', 'Total',
       'NEntregas', 'EstadoLínea', 'CentroCosto',
       'EstadoOrdenCompra', 'Clasificacion', 'Proveedor', 'FechaPreparación',
       'FechaEntrega', 'AlmacenIngreso',
        'FormadePago', 'Numero']]

In [20]:
df_ordenes_compras.to_sql('ordenes_compras_medicamentos', con=engine, schema='dbo', if_exists='replace', index=False)

87

In [59]:
nan_counts = final_df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print(nan_columns)

Comentario           38911
FechaEntrega            36
AlmacenIngreso       20197
Ncontrato            44079
NroRequerimiento     24925
CentroCosto.1        24925
CentroCostoNombre    24925
RQAprobadoPor        25149
FormadePago            109
Numero                 109
dtype: int64


In [50]:
final_df

,Companía,OrdenCompra,Item_Commodity,Descripcion,CantidadPedida,CantidadRecibida,PrecioUnitario,Total,Comentario,NEntregas,...,FechaEntrega,AlmacenIngreso,Ncontrato,UNegocio,NroRequerimiento,CentroCosto.1,CentroCostoNombre,RQAprobadoPor,FormadePago,Numero
0,1000000,5,11006,CAMPANA EXTRACTORA,1.0,0.0,19576.270000,19576.27,None,0,...,2018-05-11,LIMA - ACTIVOS FIJOS,NaN,1,NaN,NaN,None,None,L/60 Días,5.0
1,1000000,1580,11006,CAMPANA EXTRACTORA,1.0,1.0,19576.270000,19576.27,(Incluye ductería y motor extractor para cocin...,0,...,2018-07-04,None,NaN,1,NaN,NaN,None,None,Crédito,1580.0
2,1000000,20494,11006,I- PATRIMUL 20 MCG Aer.Susp.x 200 DOSIS (MEGA...,20.0,20.0,70.200000,1404.00,None,0,...,2020-12-04,LIMA - ALMACÉN MEDICINAS,NaN,1,90974.0,10808.0,DPTO. DE FARMACIA,"MENDOZA RIOJAS, EVONY LISETH",L/120 Días,20494.0
3,1000000,6,8010,ARCO EN C CIOS ALPHA,1.0,1.0,164749.423730,164749.42,None,0,...,2018-05-31,SURCO - ACTIVOS FIJOS,NaN,1,NaN,NaN,None,None,10% Adelantado - Saldo Crédito (12),6.0
4,1000000,88,8010,CORTIPREX JBE,2.0,2.0,15.439970,30.88,None,0,...,2018-05-11,LIMA - ALMACÉN MEDICINAS,NaN,1,NaN,NaN,None,None,L/90 Días,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44074,1000000,37498,7633,TROPICAMIDA 1% X 15 ML,15.0,15.0,35.644068,534.66,None,0,...,2024-04-07,LIMA - ALMACÉN MEDICINAS,NaN,1,159161.0,10808.0,DPTO. DE FARMACIA,"MENDOZA RIOJAS, EVONY LISETH",L/90 Días,37498.0
44075,1000000,37498,13580,FENILEFRINA 10% X 5 ML,15.0,15.0,40.347458,605.21,None,0,...,2024-04-07,LIMA - ALMACÉN MEDICINAS,NaN,1,159161.0,10808.0,DPTO. DE FARMACIA,"MENDOZA RIOJAS, EVONY LISETH",L/90 Días,37498.0
44076,1000000,37499,13582,VABYSMO 120 MG,1.0,1.0,2719.161017,2719.16,None,0,...,2024-04-07,LIMA - ALMACÉN MEDICINAS,NaN,1,159323.0,10808.0,DPTO. DE FARMACIA,"MENDOZA RIOJAS, EVONY LISETH",Contado,37499.0
44077,1000000,37536,6443,VOLTAREN EMULGEL 30 G,45.0,0.0,15.318000,689.31,None,0,...,2024-04-14,LIMA - ALMACÉN MEDICINAS,NaN,1,158575.0,10808.0,DPTO. DE FARMACIA,"MENDOZA RIOJAS, EVONY LISETH",L/120 Días,37536.0


In [18]:
import pandas as pd 


data = pd.read_csv('raw_cl/precios_vs.csv',encoding='UTF-8')

data.to_parquet('raw_cl/precios_vs.parquet')

data

,Descripcion,precioUnCompra,PeriodoCompra,Proveedor,precio_venta
0,ABRIMED JBE. x 120 ML (PORTUGAL),8.73,202102,FARMAGROUP S.A.C,83.16
1,ABRIMED JBE. x 120 ML (PORTUGAL),8.73,202105,FARMAGROUP S.A.C,83.16
2,ABRIMED JBE. x 120 ML (PORTUGAL),9.15,202302,FARMAGROUP S.A.C,83.16
3,ACCUPRIL 20 mg. TAB,4.24,202006,ALFARO S.A.C.,7.27
4,ACCUPRIL 20 mg. TAB,4.81,201808,ALFARO S.A.C.,7.27
...,...,...,...,...,...
9925,ZOMIPRERIN - PD 50 MG AMP (IMMACULE),84.75,202102,BOTICA FARMA SOLUCION,237.80
9926,ZOMIPRERIN - PD 50 MG AMP (IMMACULE),88.98,202103,BOTICA FARMA SOLUCION,237.80
9927,ZOMIPRERIN - PD 50 MG AMP (IMMACULE),105.93,202104,BOTICA FARMA SOLUCION,237.80
9928,ZOMIPRERIN - PD 50 MG AMP (IMMACULE),105.93,202105,BOTICA FARMA SOLUCION,237.80


In [2]:
import pandas as pd
data_parquet=pd.read_parquet('raw_cl/precios_vs.parquet')



In [3]:
data_parquet

,Descripcion,precioUnCompra,PeriodoCompra,Proveedor,precio_venta
0,ABRIMED JBE. x 120 ML (PORTUGAL),8.73,202102,FARMAGROUP S.A.C,83.16
1,ABRIMED JBE. x 120 ML (PORTUGAL),8.73,202105,FARMAGROUP S.A.C,83.16
2,ABRIMED JBE. x 120 ML (PORTUGAL),9.15,202302,FARMAGROUP S.A.C,83.16
3,ACCUPRIL 20 mg. TAB,4.24,202006,ALFARO S.A.C.,7.27
4,ACCUPRIL 20 mg. TAB,4.81,201808,ALFARO S.A.C.,7.27
...,...,...,...,...,...
9925,ZOMIPRERIN - PD 50 MG AMP (IMMACULE),84.75,202102,BOTICA FARMA SOLUCION,237.80
9926,ZOMIPRERIN - PD 50 MG AMP (IMMACULE),88.98,202103,BOTICA FARMA SOLUCION,237.80
9927,ZOMIPRERIN - PD 50 MG AMP (IMMACULE),105.93,202104,BOTICA FARMA SOLUCION,237.80
9928,ZOMIPRERIN - PD 50 MG AMP (IMMACULE),105.93,202105,BOTICA FARMA SOLUCION,237.80


In [6]:
pip install boto3


  Using cached boto3-1.34.90-py3-none-any.whl.metadata (6.6 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
   ---------------------------------------- 0.0/139.3 kB ? eta -:--:--
   ----------------- ---------------------- 61.4/139.3 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 139.3/139.3 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
    --------------------------------------- 0.3/12.2 MB 6.1 MB/s eta 0:00:02
   -- ------------------------------------- 0.8/12.2 MB 8.0 MB/s eta 0:00:02
   ---- ----------------------------------- 1.5/12.2 MB 10.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/12.2 MB 13.8 MB/s eta 0:00:01
   ------------- -------------------------- 4.2/12.2 MB 17.7 MB/s eta 0:00:01
   ------------------ --------------------- 5.7/12.2 MB 20.3 MB/s eta 0:00:01
   ---------------------- ----------------- 6.9/12.2 MB 21.9 MB/s eta 0:00:01
   -------------------